In [310]:
import pandas as pd
import numpy as np
from os import listdir

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import random
import math

In [311]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/datc/ortho/Cleaning/step2/' + PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

In [312]:

Patients = {}
testdataurl = '/datc/ortho/Cleaning/step2/'
for PatientFile in listdir(testdataurl):
    if ('meting' not in PatientFile):
        print('Skipping: %s' % PatientFile)
        # Skip this iteration
        continue
    Splitted = PatientFile.split('_')
    Patient = Splitted[0] + '_' + Splitted[1]

    Data = HeaderMaker(PatientFile)
    MaxDict = Data.max().to_dict() # .abs() for absolute numbers
    
    if(Patient not in Patients.keys()):
        Patients[Patient] = MaxDict
        Patients[Patient]['GroundTruth'] = Splitted[0].split('Cat')[1]
        Patients[Patient]['Bias'] = 0
        continue
    
    for key in MaxDict.keys():
        Patients[Patient][key]= max(Patients[Patient][key],MaxDict[key])
        

    
    
            
            
            
            

In [313]:
df = pd.DataFrame.from_dict(Patients, orient='index')
# print(df.columns.values)
df = df[['Cat3' in index or 'Cat4' in index for index in df.index]] # Welke colommen wil je houden

df['TrueFalse'] = ['4' == i for i in df['GroundTruth']]

# print(df.head())

In [314]:
Xcolumns = ['Bias']
Xcolumns.extend(['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y',
 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x',
 'humerus_r_y', 'humerus_r_z', 'ellebooghoek_r', 'thorax_l_x',
 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z',
 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y',
 'humerus_l_z', 'ellebooghoek_l'])

# X = df[Xcolumns]
# y = df[['GroundTruth']]




In [315]:
print(df['GroundTruth'].unique())

['3' '4']


In [316]:
def Patient_test_split(dataframe, xcol, ycol, Percentage):
    allIndex = np.unique(df.index.tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in df['GroundTruth'].unique(): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = np.array([i.split('_')[0].split('Cat')[1] for i in allIndex])
        CatPatients = CatPatients[CatPatients==i]
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*Percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*Percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df.index.isin(PercentageIndex)]
    Percentagey = df[ycol][df.index.isin(PercentageIndex)]
    
    Testdf = df[xcol][~df.index.isin(PercentageIndex)]
    Testy = df[ycol][~df.index.isin(PercentageIndex)]
    print('Final Percentage test: %s' % (len(Testdf)/len(Percentagedf)))
    return (Percentagedf, Testdf, Percentagey, Testy)

X_train, X_test, y_train, y_test = Patient_test_split(df, Xcolumns, 'TrueFalse', 0.85)

Final Percentage test: 0.2


In [317]:
print(len(X_train), len(X_test))

35 7


In [318]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [319]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [320]:
y_pred = lr.predict(X_test)
print(lr.coef_[0])
print(metrics.accuracy_score(y_test, y_pred))
print(y_test)

[ 0.          0.0152968  -0.02915241  0.00997657 -0.08927448 -0.00305566
 -0.02462792 -0.0537537   0.10549853  0.08645274  0.05748671 -0.1625125
  0.04485686  0.00084766  0.0152968  -0.01521135 -0.02504779 -0.07120407
  0.00227123  0.07995811 -0.05293983  0.13337942  0.03015809 -0.05717823
 -0.02731522  0.03589297 -0.00023451]
1.0
Cat3_pat10    False
Cat3_pat12    False
Cat3_pat16    False
Cat3_pat20    False
Cat3_pat33    False
Cat3_pat35    False
Cat4_pat20     True
Name: TrueFalse, dtype: bool


In [321]:
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix

y_true = y_test
y_pred =lr.predict(X_test)

TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()
# print(y_true)
# tabel printen

# print('Informationi matrix %s' % cat)
tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
print()
print("recall: ", recall_score(y_true, y_pred))
print("precision: ", precision_score(y_true, y_pred))
print("accuracy: ", accuracy_score(y_true, y_pred))
print('\n\n')

             pos  neg
0  pred pos    1    0
1  pred neg    0    6

recall:  1.0
precision:  1.0
accuracy:  1.0





In [322]:
print(metrics.classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

       False      1.000     1.000     1.000         6
        True      1.000     1.000     1.000         1

   micro avg      1.000     1.000     1.000         7
   macro avg      1.000     1.000     1.000         7
weighted avg      1.000     1.000     1.000         7

